In [1]:
from dask.distributed import Client, progress
# HPC
# client = Client(scheduler_file='/glade/scratch/jhamman/scheduler.json')
# client

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:    tcp://10.32.2.28:38589
distributed.scheduler - INFO -   dashboard at:                     :8787


In [2]:
client = Client(cluster)
client

distributed.scheduler - INFO - Receive client connection: Client-0721e158-6363-11ea-83b1-965f2ad2986c
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.32.2.28:38589 Dashboard: /user/0000-0001-7783-5629/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
import sys, glob
import numpy as np
import xarray as xr
import xscale.spectral.fft as xfft
import xscale 
import Wavenum_freq_spec_func as wfs
import time

In [4]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
from matplotlib.colors import LogNorm

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic


In [5]:
%%time
import gcsfs

fs = gcsfs.GCSFileSystem(project='pangeo-181919', token='anon', access='read_only')

gcsmapgridu=gcsfs.mapping.GCSMap('pangeo-data/NATL60-CJM165-SSU-1h-1m2deg2deg',gcs=fs,check=False,create=False)
dsu=xr.open_zarr(gcsmapgridu)

gcsmapgridv=gcsfs.mapping.GCSMap('pangeo-data/NATL60-CJM165-SSV-1h-1m2deg2deg',gcs=fs,check=False,create=False)
dsv=xr.open_zarr(gcsmapgridv)


distributed.scheduler - INFO - Register tcp://10.32.13.6:44869
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.13.6:44869
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.27.3:44249
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.27.3:44249
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.5.12:36199
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.5.12:36199
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.26.9:33721
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.26.9:33721
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.18.7:42659
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.18.7:42659
distributed.core - INFO

CPU times: user 38.1 s, sys: 4.49 s, total: 42.6 s
Wall time: 2min 13s


In [6]:
%%time
lat=dsu['nav_lat']
lon=dsu['nav_lon']
 
latmin = 40.0; latmax = 45.0;
lonmin = -40.0; lonmax = -35.0;

domain = (lonmin<lon) * (lon<lonmax) * (latmin<lat) * (lat<latmax)
where = np.where(domain)

#get indice
jmin = np.min(where[0][:])
jmax = np.max(where[0][:])
imin = np.min(where[1][:])
imax = np.max(where[1][:])

latbox=lat[jmin:jmax,imin:imax]
lonbox=lon[jmin:jmax,imin:imax]


CPU times: user 772 ms, sys: 83.7 ms, total: 856 ms
Wall time: 4.91 s


In [ ]:
%%time
print('Select dates')
u_JFM=dsu.sel(time_counter=slice('2013-01-01','2013-03-31'))['vozocrtx']
v_JFM=dsv.sel(time_counter=slice('2013-01-01','2013-03-31'))['vomecrty']

print('Select box area')
u_JFM_box=u_JFM[:,jmin:jmax,imin:imax].chunk(chunks={'time_counter':24,'x':120,'y':120})
v_JFM_box=v_JFM[:,jmin:jmax,imin:imax].chunk(chunks={'time_counter':24,'x':120,'y':120})

# - get dx and dy
dx_JFM,dy_JFM = wfs.get_dx_dy(u_JFM_box[0],lonbox,latbox)


#... Detrend data in all dimension ...
print('Detrend data in all dimension')
u_JFM = wfs.detrendn(u_JFM_box,axes=[0,1,2])
v_JFM = wfs.detrendn(v_JFM_box,axes=[0,1,2])

#... Apply hanning windowing ...') 
print('Apply hanning windowing')
u_JFM = wfs.apply_window(u_JFM, u_JFM.dims, window_type='hanning')
v_JFM = wfs.apply_window(v_JFM, v_JFM.dims, window_type='hanning')

# - get derivatives
derivatives_JFM = wfs.velocity_derivatives(u_JFM, v_JFM, xdim='x', ydim='y', dx={'x': dx_JFM, 'y': dy_JFM})
dudx_JFM = derivatives_JFM['u_x']; dudy_JFM = derivatives_JFM['u_y']
dvdx_JFM = derivatives_JFM['v_x']; dvdy_JFM = derivatives_JFM['v_y']

# - compute terms
phi1_JFM = u_JFM*dudx_JFM + v_JFM*dudy_JFM
phi2_JFM = u_JFM*dvdx_JFM + v_JFM*dvdy_JFM

u_JFMhat = xfft.fft(u_JFM, dim=('time_counter', 'x', 'y'), dx={'x': dx_JFM, 'y': dx_JFM}, sym=True)
v_JFMhat = xfft.fft(v_JFM, dim=('time_counter', 'x', 'y'), dx={'x': dx_JFM, 'y': dx_JFM}, sym=True)

phi1_JFM_hat = xfft.fft(phi1_JFM, dim=('time_counter', 'x', 'y'), dx={'x': dx_JFM, 'y': dx_JFM}, sym=True)
phi2_JFM_hat = xfft.fft(phi2_JFM, dim=('time_counter', 'x', 'y'), dx={'x': dx_JFM, 'y': dx_JFM}, sym=True)

tm1_JFM = (u_JFMhat.conj())*phi1_JFM_hat
tm2_JFM = (v_JFMhat.conj())*phi2_JFM_hat

# - computer transfer
Nk_JFM,Nj_JFM,Ni_JFM = u_JFM.shape
transfer_2D_JFM = -1.0*(tm1_JFM + tm2_JFM)/np.square(Ni_JFM*Nj_JFM)
transfer_term_JFM = transfer_2D_JFM.real

#... Get frequency and wavenumber ... 
print('Get frequency and wavenumber')
ffrequency_JFM = u_JFMhat.f_time_counter
kx_JFM = u_JFMhat.f_x
ky_JFM = u_JFMhat.f_y

#... Get istropic wavenumber ... 
print('Get istropic wavenumber')
wavenumber_JFM,kradial_JFM = wfs.get_wavnum_kradial(kx_JFM,ky_JFM)

#... Get numpy array ... 
print('Get numpy array')
var_psd_np_JFM = transfer_term_JFM.values

#... Get 2D frequency-wavenumber field ... 
print('Get transfer')
transfer_JFM = wfs.get_f_k_in_2D(kradial_JFM,wavenumber_JFM,var_psd_np_JFM) 

print('Get flux')
flux_JFM = wfs.get_flux_in_1D(kradial_JFM,wavenumber_JFM,var_psd_np_JFM)


Select dates
Select box area
Detrend data in all dimension


In [ ]:
# Save to Netscdf file
# - build dataarray
print('Save to Netscdf file')
transfer_JFM_da = xr.DataArray(transfer_JFM,dims=['frequency','wavenumber'],name="transfer",coords=[ffrequency_JFM ,wavenumber_JFM])
flux_JFM_da = xr.DataArray(flux_JFM,dims=['frequency','wavenumber'],name="flux",coords=[ffrequency_JFM,wavenumber_JFM])
transfer_JFM_da.attrs['Name'] = 'KE_Transfer_Flux_JFM_w_k_from_1h_NATL60-CJM165.nc'

transfer_JFM_da.to_dataset().to_netcdf(path='KE_Transfer_Flux_JFM_w_k_from_1h_NATL60-CJM165.nc',mode='w',engine='scipy')
flux_JFM_da.to_dataset().to_netcdf(path='KE_Transfer_Flux_JFM_w_k_from_1h_NATL60-CJM165.nc',mode='a',engine='scipy')


In [ ]:
%%time
print('Select dates')
u_JAS=dsu.sel(time_counter=slice('2013-06-01','2013-09-30'))['vozocrtx']
v_JAS=dsv.sel(time_counter=slice('2013-06-01','2013-09-30'))['vomecrty']

print('Select box area')
u_JAS_box=u_JAS[:,jmin:jmax,imin:imax].chunk(chunks={'time_counter':1000,'x':120,'y':120})

v_JAS_box=v_JAS[:,jmin:jmax,imin:imax].chunk(chunks={'time_counter':1000,'x':120,'y':120})


# - get dx and dy
dx_JAS,dy_JAS = wfs.get_dx_dy(u_JAS_box[0],lonbox,latbox)


#... Detrend data in all dimension ...
print('Detrend data in all dimension')
u_JAS = wfs.detrendn(u_JAS_box,axes=[0,1,2])
v_JAS = wfs.detrendn(v_JAS_box,axes=[0,1,2])

#... Apply hanning windowing ...') 
print('Apply hanning windowing')
u_JAS = wfs.apply_window(u_JAS, u_JAS.dims, window_type='hanning')
v_JAS = wfs.apply_window(v_JAS, v_JAS.dims, window_type='hanning')

# - get derivatives
print('velocity derivatives')
derivatives_JAS = wfs.velocity_derivatives(u_JAS, v_JAS, xdim='x', ydim='y', dx={'x': dx_JAS, 'y': dy_JAS})
dudx_JAS = derivatives_JAS['u_x']; dudy_JAS = derivatives_JAS['u_y']
dvdx_JAS = derivatives_JAS['v_x']; dvdy_JAS = derivatives_JAS['v_y']

# - compute terms
print('computer terms')
phi1_JAS = u_JAS*dudx_JAS + v_JAS*dudy_JAS
phi2_JAS = u_JAS*dvdx_JAS + v_JAS*dvdy_JAS

print('fft u v ')
u_JAShat = xfft.fft(u_JAS, dim=('time_counter', 'x', 'y'), dx={'x': dx_JAS, 'y': dx_JAS}, sym=True)
v_JAShat = xfft.fft(v_JAS, dim=('time_counter', 'x', 'y'), dx={'x': dx_JAS, 'y': dx_JAS}, sym=True)

print('fft phi')
phi1_JAS_hat = xfft.fft(phi1_JAS, dim=('time_counter', 'x', 'y'), dx={'x': dx_JAS, 'y': dx_JAS}, sym=True)
phi2_JAS_hat = xfft.fft(phi2_JAS, dim=('time_counter', 'x', 'y'), dx={'x': dx_JAS, 'y': dx_JAS}, sym=True)

print('multiply')
tm1_JAS = (u_JAShat.conj())*phi1_JAS_hat
tm2_JAS = (v_JAShat.conj())*phi2_JAS_hat

# - computer transfer
print('Compute transfer')
Nk_JAS,Nj_JAS,Ni_JAS = u_JAS.shape
transfer_2D_JAS = -1.0*(tm1_JAS + tm2_JAS)/np.square(Ni_JAS*Nj_JAS)
transfer_term_JAS = transfer_2D_JAS.real

#... Get frequency and wavenumber ... 
print('Get frequency and wavenumber')
ffrequency_JAS = u_JAShat.f_time_counter
kx_JAS = u_JAShat.f_x
ky_JAS = u_JAShat.f_y

#... Get istropic wavenumber ... 
print('Get istropic wavenumber')
wavenumber_JAS,kradial_JAS = wfs.get_wavnum_kradial(kx_JAS,ky_JAS)



In [ ]:
u_JAShat.min()

In [ ]:
transfer_2D_JAS.min()

In [ ]:
test=transfer_term_JAS.chunk(chunks={'f_time_counter':1000,'f_y':400,'f_x':300})

In [ ]:
test.max()

In [ ]:
var_psd_np_JAS = test.values

In [ ]:
var_psd_np_JAS.min()

In [ ]:
#... Get numpy array ... 
#print('Get numpy array')
#var_psd_np_JAS = transfer_term_JAS.values

#... Get 2D frequency-wavenumber field ... 
print('Get transfer')
transfer_JAS = wfs.get_f_k_in_2D(kradial_JAS,wavenumber_JAS,var_psd_np_JAS) 

print('Get flux')
flux_JAS = wfs.get_flux_in_1D(kradial_JAS,wavenumber_JAS,var_psd_np_JAS)


In [ ]:
# Save to Netscdf file
# - build dataarray
print('Save to Netscdf file')
transfer_JAS_da = xr.DataArray(transfer_JAS,dims=['frequency','wavenumber'],name="transfer",coords=[ffrequency_JAS ,wavenumber_JAS])
flux_JAS_da = xr.DataArray(flux_JAS,dims=['frequency','wavenumber'],name="flux",coords=[ffrequency_JAS,wavenumber_JAS])
transfer_JAS_da.attrs['Name'] = 'KE_Transfer_Flux_JAS_w_k_from_1h_NATL60-CJM165.nc'

transfer_JAS_da.to_dataset().to_netcdf(path='KE_Transfer_Flux_JAS_w_k_from_1h_NATL60-CJM165.nc',mode='w',engine='scipy')
flux_JAS_da.to_dataset().to_netcdf(path='KE_Transfer_Flux_JAS_w_k_from_1h_NATL60-CJM165.nc',mode='a',engine='scipy')
